In [ ]:
!pip install polyleven

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from collections import Counter
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster import hierarchy
from scipy.cluster.hierarchy import single
from scipy.cluster.hierarchy import fcluster 
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import linkage
from sklearn.cluster import DBSCAN
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import leaders
import sys
import operator
from polyleven import levenshtein


country='England'
data=pd.read_csv(f'../DATASET/England_complete.csv',index_col=False)
data=data.drop(columns='Unnamed: 0')
mask=[True if len(x)==1273 else False for x in data['Aligned'].values]
data=data[mask]
data

dataset_cluster=pd.DataFrame()
n_cl=[]
n_ds=[]
perc=[]
cnt=1
distances=[]
max_month=max(data['Month'])
for month in range(3,max_month+1):
    #print(month)
    
    temp=data[data['Month']==month]
    temp=temp[[True if 'X' not in seq else False for seq in temp['Sequence']]]
    seq=pd.read_csv(f'../DISTANCE MATRICES/{month}_seq_list.csv')
    seq=list(seq['0'].values)
    mask=[True if 'X' not in el else False for el in seq]
    seq=np.array(seq)[mask]
    path= '../DISTANCE MATRICES/' + str(month) + '.npy'
    matrix_com=np.load(path)
    
    posizioni=[]
    for el in matrix_com:
        posizioni.append(list(el).index(0))
    posizioni=np.array(posizioni)
    accepted_pos=posizioni[:len(mask)][mask]

    mask=[True if pos in accepted_pos else False for pos in posizioni]
    matrix_com=matrix_com[mask,:][:,mask]
    posizioni=posizioni[mask]
    #print(len(posizioni))
    #print(np.shape(matrix_com))
    Nb=np.shape(matrix_com)[0]
    sup=Nb*1/100
   
    matrix=squareform(matrix_com)
    Z = linkage(matrix, method='ward')
    th=100
        
    a=fcluster(Z,t=th,criterion='distance')
    hh=Counter(a)
    cluster=np.zeros(len(seq))
    L=[]
    V=[]
    print(f'In month {month}')
    clust_count={}
    for k,v in hh.items():
        if v>sup:
            #print(k,v)
            L.append(k)
            V.append(v)
            mask=np.array(a)==k
            elementi=posizioni[mask]
            hh=Counter(elementi)
            ds=max(hh.items(), key=operator.itemgetter(1))[0]
            #print(ds)
            distanze=matrix_com[ds][:len(seq)][mask[:len(seq)]]
            
            cluster[mask[:len(seq)]]=cnt
            clust_count[k]=cnt
            print(f'Cluster number {cnt} represents {round(v/Nb,2)} of the dataset, with {len(hh.keys())} different sequences, statistics : {max(distanze)}')#{round(np.mean(distanze),2)} and {round(np.std(distanze),2)}')
            cnt=cnt+1
            distances.append(max(distanze))
    #print(Counter(cluster))
    #print(cluster)
    cluster_new=np.copy(cluster)
   
    for i,el in enumerate(cluster):
        if el ==0:
            riga=matrix_com[i,:]
            distanze=[]
            for clu in L:
                mask=np.array(a)==clu
                distanze.append(np.mean(riga[mask]))
            index=distanze.index(min(distanze))
            cluster_new[i]=clust_count[L[index]]
            
            
        
    #print(Counter(cluster_new))
    cluster=np.copy(cluster_new)
    cluster_col=np.zeros(len(temp))
    for i,el in enumerate(cluster):
        target=seq[i]
        mask=temp['Sequence']==target
        cluster_col[mask]=el
    temp['Cluster']=np.copy(cluster_col)
   
        
    #print(sum(temp['Cluster']==0))
    dataset_cluster=pd.concat([dataset_cluster,temp])
    n_cl.append(len(V))
    perc.append((sum(V)/len(posizioni)))
   
    print()
    

import os
#os.mkdir('DATASET')
dataset_cluster.to_csv('DATASET/data_with_clusters.csv')